# Bikeshare Submission

## Packages Used

In [1]:
import sys
sys.path.append('..')

import pandas as pd
import os
from src import DataLoader

You are provided hourly rental data spanning two years. For this competition, the training set is comprised of the first 19 days of each month, while the test set is the 20th to the end of the month. You must predict the total count of bikes rented during each hour covered by the test set, using only information available prior to the rental period.

Data Fields
- datetime - hourly date + timestamp  
- season 
    - 1 = spring, 
    - 2 = summer, 
    - 3 = fall, 
    - 4 = winter 
- holiday - whether the day is considered a holiday
- workingday - whether the day is neither a weekend nor holiday
- weather 
    - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
    - 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
    - 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
    - 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 
- temp - temperature in Celsius
- atemp - "feels like" temperature in Celsius
- humidity - relative humidity
- windspeed - wind speed
- casual - number of non-registered user rentals initiated
- registered - number of registered user rentals initiated
- count - number of total rentals <- PREDICTION

# Training without Using Data Engineering

In [2]:
from autogluon.tabular import TabularPredictor

/Users/samuel/anaconda3/envs/bikeshare/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Extracting Training and Testing Data

In [3]:
loader = DataLoader()

# Loading raw trained and test data to process without any changes
train_df, test_df = loader.get_raw_train_test_data()


        Data Loader initialized with data path: {self.data_path}
            - Use load_raw() to load raw data
            - Use get_train_test_data() to get raw train and test data  
            - Use save_feature_engineered_data() to save feature engineered data
            - Use load_feature_engineered() to load feature engineered data
              
            - Use set_as_category() to set columns as category
        


AttributeError: 'DataLoader' object has no attribute 'get_raw_train_test_data'

## - Training without New Features

In [ ]:
predictor = TabularPredictor(
        label='count',
        path='autogluon',
        eval_metric='root_mean_squared_error',
    ).fit(
        train_df,
        time_limit=600,
        presets='best_quality'
    )

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.8
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.5.0: Wed May  1 20:12:58 PDT 2024; root:xnu-10063.121.3~5/RELEASE_ARM64_T6000
CPU Count:          8
Memory Avail:       2.72 GB / 16.00 GB (17.0%)
Disk Space Avail:   22.01 GB / 460.43 GB (4.8%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then

[1000]	valid_set's rmse: 5.59844
[2000]	valid_set's rmse: 5.06757
[3000]	valid_set's rmse: 4.87535


	Ran out of time, early stopping on iteration 3187. Best iteration is:
	[3162]	valid_set's rmse: 4.84423


[1000]	valid_set's rmse: 5.59925
[2000]	valid_set's rmse: 5.06547
[3000]	valid_set's rmse: 4.89408


	Ran out of time, early stopping on iteration 3788. Best iteration is:
	[3788]	valid_set's rmse: 4.83114


...

	Ran out of time, early stopping on iteration 5857. Best iteration is:
	[5800]	valid_set's rmse: 3.99784


[1000]	valid_set's rmse: 4.18392


	Ran out of time, early stopping on iteration 1365. Best iteration is:
	[1364]	valid_set's rmse: 4.07552


[1000]	valid_set's rmse: 4.47659
[2000]	valid_set's rmse: 4.21038
[3000]	valid_set's rmse: 4.10809
[4000]	valid_set's rmse: 4.05249
[5000]	valid_set's rmse: 4.02098
[6000]	valid_set's rmse: 3.99483
[7000]	valid_set's rmse: 3.97811


	Ran out of time, early stopping on iteration 7923. Best iteration is:
	[7915]	valid_set's rmse: 3.95948
	-4.2697	 = Validation score   (-root_mean_squared_error)
	140.91s	 = Training   runtime
	0.61s	 = Validation runtime
Fitting model: LightGBM_BAG_L2 ... Training model for up to 5.05s of the 5.03s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	Ran out of time, early stopping on iteration 125. Best iteration is:
	[124]	valid_set's rmse: 3.15529
	Ran out of time, early stopping on iteration 116. Best iteration is:
	[116]	valid_set's rmse: 3.57582
	Ran out of time, early stopping on iteration 132. Best iteration is:
	[132]	valid_set's rmse: 2.55042
	Ran out of time, early stopping on iteration 121. Best iteration is:
	[121]	valid_set's rmse: 3.20589
	Ran out of time, early stopping on iteration 153. Best iteration is:
	[153]	valid_set's rmse: 3.07068
	Ran out of time, early stopping on iteration 161. Best iteration is:
	[159]

## - Showing the Leatherboard

In [ ]:
try:
    display(predictor.leaderboard(silent=True))
except:
    predictor = TabularPredictor.load('autogluon')
    display(predictor.leaderboard(silent=True))

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-54.569714,root_mean_squared_error,1.292876,443.947090,0.000248,0.011255,3,True,9
1,LightGBM_BAG_L2,-54.586355,root_mean_squared_error,1.010519,318.299719,0.039332,20.634286,2,True,8
2,LightGBMXT_BAG_L2,-60.570375,root_mean_squared_error,1.253296,423.301549,0.282109,125.636116,2,True,7
3,KNeighborsDist_BAG_L1,-84.146423,root_mean_squared_error,0.014645,0.009696,0.014645,0.009696,1,True,2
4,WeightedEnsemble_L2,-84.146423,root_mean_squared_error,0.014847,0.021625,0.000202,0.011929,2,True,6
5,KNeighborsUnif_BAG_L1,-101.588176,root_mean_squared_error,0.014143,0.006946,0.014143,0.006946,1,True,1
6,RandomForestMSE_BAG_L1,-116.542032,root_mean_squared_error,0.241228,1.870387,0.241228,1.870387,1,True,5
7,LightGBM_BAG_L1,-131.432362,root_mean_squared_error,0.088231,39.867387,0.088231,39.867387,1,True,4
8,LightGBMXT_BAG_L1,-131.494795,root_mean_squared_error,0.612940,255.911016,0.612940,255.911016,1,True,3


## - Predictions on Test Data

In [ ]:
predictions = predictor.predict(test_df)
predictions.head()

0    34.273094
1    43.230057
2    46.590622
3    50.979973
4    51.777607
Name: count, dtype: float32

In [ ]:
# Identifying negative predictions
predictions.describe()

count    6493.000000
mean       98.326591
std        88.796036
min       -10.711480
25%        16.453503
50%        62.550030
75%       169.153320
max       370.938568
Name: count, dtype: float64

In [ ]:
# Counting negative predictions
negative_prediction_count = (predictions < 0).sum()

print(f"There are {negative_prediction_count} negative predictions to set to zero.")

There are 9 negative predictions to set to zero.


## - Setting up for Submission

In [ ]:
# Setting negative predictions to zero
predictions[predictions < 0] = 0

predictions.describe()

count    6493.000000
mean       98.332489
std        88.789307
min         0.000000
25%        16.453503
50%        62.550030
75%       169.153320
max       370.938568
Name: count, dtype: float64

In [ ]:
submission_df = pd.read_csv('../data/sampleSubmission.csv')
display(submission_df.head())
submission_df.shape

,datetime,count
0,2011-01-20 00:00:00,0
1,2011-01-20 01:00:00,0
2,2011-01-20 02:00:00,0
3,2011-01-20 03:00:00,0
4,2011-01-20 04:00:00,0


(6493, 2)

In [ ]:
submission_df['count'] = predictions
submission_df.to_csv('submission.csv', index=False)
display(submission_df.head())
submission_df.shape

,datetime,count
0,2011-01-20 00:00:00,34.273094
1,2011-01-20 01:00:00,43.230057
2,2011-01-20 02:00:00,46.590622
3,2011-01-20 03:00:00,50.979973
4,2011-01-20 04:00:00,51.777607


(6493, 2)

## - Submitting Initial Predictions

In [ ]:
# Getting the best model name
best_model = predictor.model_best
best_model

'WeightedEnsemble_L3'

In [ ]:
!kaggle competitions submit -c bike-sharing-demand -f submission.csv -m f"irst submission with {best_model}"

100%|█████████████████████████████████████████| 188k/188k [00:00<00:00, 436kB/s]
Successfully submitted to Bike Sharing Demand

In [ ]:
!kaggle competitions submissions -c bike-sharing-demand

fileName        date                 description                                status    publicScore  privateScore  
--------------  -------------------  -----------------------------------------  --------  -----------  ------------  
submission.csv  2024-07-07 21:14:45  first submission with WeightedEnsemble_L3  complete  1.84764      1.84764       


# Training with Feature Engineering

In [1]:
import sys
sys.path.append('..')

from src import DataLoader

## - Extract Feature Engineered Data

In [2]:
loader = DataLoader()
loader.load_raw()

# Loading raw trained and test data with changes to the season and weather columns
loader.set_as_category(columns=["season", "weather"])
train_df, test_df = loader.get_train_test_data()


        Data Loader initialized with data path: ../data/
            - Use load_raw() to load raw data
            - Use get_train_test_data() to get raw train and test data  
            - Use save_feature_engineered_data() to save feature engineered data
            - Use load_feature_engineered() to load feature engineered data
              
            - Use set_as_category() to set columns as category

        Checkpoints available: ['hyperparameter_tuning']
        
Columns ['season', 'weather'] set as category successfully!


In [3]:
train_df.dtypes

datetime        object
season        category
holiday          int64
workingday       int64
weather       category
temp           float64
atemp          float64
humidity         int64
windspeed      float64
casual           int64
registered       int64
count            int64
dtype: object

## - Re-training the model with categorical columns

In [ ]:
predictor_new_features = TabularPredictor(
    label='count',
    path='autogluon-new-features',
    eval_metric='root_mean_squared_error',
).fit(
    train_df,
    time_limit=600,
    presets='best_quality'
)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.8
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.5.0: Wed May  1 20:12:58 PDT 2024; root:xnu-10063.121.3~5/RELEASE_ARM64_T6000
CPU Count:          8
Memory Avail:       3.77 GB / 16.00 GB (23.6%)
Disk Space Avail:   38.90 GB / 460.43 GB (8.4%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then

[1000]	valid_set's rmse: 129.884
[2000]	valid_set's rmse: 129.338
[1000]	valid_set's rmse: 129.41
[2000]	valid_set's rmse: 128.542
[1000]	valid_set's rmse: 132.736
[2000]	valid_set's rmse: 132.145
[1000]	valid_set's rmse: 126.902
[2000]	valid_set's rmse: 125.608
[3000]	valid_set's rmse: 125.013
[1000]	valid_set's rmse: 134.952
[2000]	valid_set's rmse: 134.183
[3000]	valid_set's rmse: 133.455
[4000]	valid_set's rmse: 133.177
[5000]	valid_set's rmse: 132.541
[6000]	valid_set's rmse: 132.433


	Ran out of time, early stopping on iteration 6371. Best iteration is:
	[6303]	valid_set's rmse: 132.303


[1000]	valid_set's rmse: 137.272
[2000]	valid_set's rmse: 135.531
[3000]	valid_set's rmse: 134.963
[4000]	valid_set's rmse: 134.68


	Ran out of time, early stopping on iteration 4317. Best iteration is:
	[4013]	valid_set's rmse: 134.644


...

	-131.3015	 = Validation score   (-root_mean_squared_error)
	252.02s	 = Training   runtime
	0.45s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ... Training model for up to 45.89s of the 195.67s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	Ran out of time, early stopping on iteration 432. Best iteration is:
	[426]	valid_set's rmse: 130.478


[1000]	valid_set's rmse: 133.028
[1000]	valid_set's rmse: 131.187
[1000]	valid_set's rmse: 126.598
[1000]	valid_set's rmse: 130.79
[2000]	valid_set's rmse: 130.757
[1000]	valid_set's rmse: 133.168
[1000]	valid_set's rmse: 131.157
[1000]	valid_set's rmse: 130.666


	-130.7397	 = Validation score   (-root_mean_squared_error)
	34.12s	 = Training   runtime
	0.12s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L1 ... Training model for up to 11.3s of the 161.07s of remaining time.
	-116.6309	 = Validation score   (-root_mean_squared_error)
	1.66s	 = Training   runtime
	0.24s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ... Training model for up to 9.15s of the 158.92s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	Ran out of time, early stopping on iteration 375.
	Ran out of time, early stopping on iteration 414.
	Ran out of time, early stopping on iteration 422.
	Ran out of time, early stopping on iteration 427.
	Ran out of time, early stopping on iteration 417.
	Ran out of time, early stopping on iteration 469.
	Ran out of time, early stopping on iteration 512.
	Ran out of time, early stopping on iteration 613.
	-136.1294	 = Validation score   (-root_mean_squared_error)


[1000]	valid_set's rmse: 61.5997
[2000]	valid_set's rmse: 60.5465
[1000]	valid_set's rmse: 60.9525
[2000]	valid_set's rmse: 59.5735
[1000]	valid_set's rmse: 63.9217
[2000]	valid_set's rmse: 62.6126
[1000]	valid_set's rmse: 63.5466
[2000]	valid_set's rmse: 62.6229
[1000]	valid_set's rmse: 58.3412
[2000]	valid_set's rmse: 57.0796
[1000]	valid_set's rmse: 62.8921
[2000]	valid_set's rmse: 61.8704
[3000]	valid_set's rmse: 62.0382
[1000]	valid_set's rmse: 64.4288
[2000]	valid_set's rmse: 63.7549
[1000]	valid_set's rmse: 58.7149
[2000]	valid_set's rmse: 58.0931


	-60.7416	 = Validation score   (-root_mean_squared_error)
	65.28s	 = Training   runtime
	0.28s	 = Validation runtime
Fitting model: LightGBM_BAG_L2 ... Training model for up to 82.75s of the 82.73s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-54.6821	 = Validation score   (-root_mean_squared_error)
	16.15s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L2 ... Training model for up to 66.5s of the 66.48s of remaining time.
	-53.0126	 = Validation score   (-root_mean_squared_error)
	5.13s	 = Training   runtime
	0.28s	 = Validation runtime
Fitting model: CatBoost_BAG_L2 ... Training model for up to 60.88s of the 60.86s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	Ran out of time, early stopping on iteration 2369.
	-55.2169	 = Validation score   (-root_mean_squared_error)
	49.99s	 = Training   runtime
	0.02s	 = Validation run

## - Showing the Leatherboard

In [ ]:
try:
    display(predictor_new_features.leaderboard(silent=True))
except:
    predictor_new_features = TabularPredictor.load('autogluon-new-features')
    display(predictor_new_features.leaderboard(silent=True))

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-52.509594,root_mean_squared_error,1.734140,369.858831,0.000209,0.014574,3,True,15
1,RandomForestMSE_BAG_L2,-53.012627,root_mean_squared_error,1.400775,302.380536,0.279177,5.125411,2,True,11
2,ExtraTreesMSE_BAG_L2,-53.647904,root_mean_squared_error,1.397926,298.586987,0.276328,1.331862,2,True,13
3,LightGBM_BAG_L2,-54.682144,root_mean_squared_error,1.162112,313.400169,0.040514,16.145044,2,True,10
4,CatBoost_BAG_L2,-55.216856,root_mean_squared_error,1.137912,347.241940,0.016314,49.986816,2,True,12
5,LightGBMXT_BAG_L2,-60.741570,root_mean_squared_error,1.396845,362.531581,0.275247,65.276457,2,True,9
6,NeuralNetFastAI_BAG_L2,-82.408300,root_mean_squared_error,1.325884,305.257513,0.204286,8.002388,2,True,14
7,KNeighborsDist_BAG_L1,-84.146423,root_mean_squared_error,0.014512,0.005741,0.014512,0.005741,1,True,2
8,WeightedEnsemble_L2,-84.146423,root_mean_squared_error,0.015391,0.016189,0.000879,0.010448,2,True,8
9,KNeighborsUnif_BAG_L1,-101.588176,root_mean_squared_error,0.015999,0.006527,0.015999,0.006527,1,True,1


## - Predictions on Test Data

In [ ]:
predictions_new_features = predictor_new_features.predict(test_df)

## - Setting up for Submission

In [ ]:
# Setting negative predictions to zero
predictions_new_features[predictions_new_features < 0] = 0

In [ ]:
submission_new_features_df = pd.read_csv('../data/sampleSubmission.csv')
submission_new_features_df['count'] = predictions_new_features
submission_new_features_df.to_csv('submission-new-features.csv', index=False)

In [ ]:
# Compare old and new submissions
submission_df = pd.read_csv('submission.csv')
submission_new_features_df = pd.read_csv('submission-new-features.csv')

submission_df.merge(submission_new_features_df, on='datetime', suffixes=('_old', '_new'))

,datetime,count_old,count_new
0,2011-01-20 00:00:00,34.273094,23.911224
1,2011-01-20 01:00:00,43.230057,42.881252
2,2011-01-20 02:00:00,46.590622,47.045166
3,2011-01-20 03:00:00,50.979973,49.280098
4,2011-01-20 04:00:00,51.777607,51.790570
...,...,...,...
6488,2012-12-31 19:00:00,168.921830,158.083010
6489,2012-12-31 20:00:00,168.921830,158.083010
6490,2012-12-31 21:00:00,169.793330,148.362730
6491,2012-12-31 22:00:00,167.674060,144.396210


## - Submitting New Predictions

In [ ]:
best_model = predictor_new_features.model_best
best_model

'WeightedEnsemble_L3'

In [ ]:
!kaggle competitions submit -c bike-sharing-demand -f submission-new-features.csv -m "new features with {best_model}"

100%|█████████████████████████████████████████| 188k/188k [00:00<00:00, 436kB/s]
Successfully submitted to Bike Sharing Demand

In [ ]:
!kaggle competitions submissions -c bike-sharing-demand

fileName                     date                 description                                status    publicScore  privateScore  
---------------------------  -------------------  -----------------------------------------  --------  -----------  ------------  
submission-new-features.csv  2024-07-08 23:47:44  new features with WeightedEnsemble_L3      complete  1.7966       1.7966        
submission.csv               2024-07-07 21:14:45  first submission with WeightedEnsemble_L3  complete  1.84764      1.84764       


# Training with Hyperparameter Tuning
The following documentation show how to train a model with hyperparameter tuning using AutoGluon.

Documentation: https://auto.gluon.ai/stable/api/autogluon.tabular.TabularPredictor.fit.html

In [4]:
import sys
sys.path.append('..')

from src import DataLoader

## - Transforming Features as Categories

In [5]:
# Converting to categorical for better performance
loader = DataLoader()

loader.load_feature_engineered(checkpoint_name='hyperparameter_tuning')
loader.set_as_category(columns=["season", "weather"])

train_df, test_df = loader.get_train_test_data()

train_df.dtypes


        Data Loader initialized with data path: ../data/
            - Use load_raw() to load raw data
            - Use get_train_test_data() to get raw train and test data  
            - Use save_feature_engineered_data() to save feature engineered data
            - Use load_feature_engineered() to load feature engineered data
              
            - Use set_as_category() to set columns as category

        Checkpoints available: ['hyperparameter_tuning']
        
Columns ['season', 'weather'] set as category successfully!


season         category
holiday           int64
workingday        int64
weather        category
temp            float64
humidity          int64
windspeed       float64
count             int64
day_of_week       int64
hour              int64
minute            int64
date             object
dtype: object

## - Creating a Validation Set for Local Scoring

In [10]:
from sklearn.model_selection import train_test_split

train_val_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

print(f""" 
Train shape: {train_val_df.shape}
Validation shape: {val_df.shape}    
""")

train_val_df.head()

 
Train shape: (8708, 12)
Validation shape: (2178, 12)    



,season,holiday,workingday,weather,temp,humidity,windspeed,count,day_of_week,hour,minute,date
2815,3,0,1,1,27.88,83,6.0032,35,2,5,0,2011-07-06
8695,3,0,0,1,36.90,39,19.9995,450,5,16,0,2012-08-04
8406,3,0,1,1,32.80,41,16.9979,276,2,15,0,2012-07-11
1543,2,0,0,2,14.76,93,7.0015,5,6,4,0,2011-04-10
4952,4,0,0,1,13.12,45,16.9979,200,5,10,0,2011-11-19


In [7]:
val_df.head()

,season,holiday,workingday,weather,temp,humidity,windspeed,count,day_of_week,hour,minute,date
3133,3,0,1,1,33.62,59,0.0000,127,1,11,0,2011-07-19
5786,1,1,0,1,4.10,54,6.0032,13,0,6,0,2012-01-16
5224,4,0,0,1,9.84,48,12.9980,163,6,18,0,2011-12-11
8953,3,0,1,2,29.52,62,12.9980,233,2,10,0,2012-08-15
8054,2,0,1,1,25.42,53,16.9979,222,4,23,0,2012-06-15


In [8]:
test_df.head()

,season,holiday,workingday,weather,temp,humidity,windspeed,day_of_week,hour,minute,date
0,1,0,1,1,10.66,56,26.0027,3,0,0,2011-01-20
1,1,0,1,1,10.66,56,0.0000,3,1,0,2011-01-20
2,1,0,1,1,10.66,56,0.0000,3,2,0,2011-01-20
3,1,0,1,1,10.66,56,11.0014,3,3,0,2011-01-20
4,1,0,1,1,10.66,56,11.0014,3,4,0,2011-01-20


## - Re-train with One Hot Encoding & Hyperparameter Tuning

In [ ]:
from autogluon.tabular.configs.hyperparameter_configs import get_hyperparameter_config
from autogluon.tabular import TabularPredictor
from autogluon.common import space

In [ ]:
hyperparameters = {
    'NN_TORCH': {'num_epochs': 10, 'activation': 'relu', 'dropout_prob': space.Real(0.0, 0.5)},
    'GBM': {'num_boost_round': 1000, 'learning_rate': space.Real(0.01, 0.1, log=True)},
    'XGB': {'n_estimators': 1000, 'learning_rate': space.Real(0.01, 0.1, log=True)}
}


hyper_timeout = 1 * 60  # seconds
time_limit = 3 * 60
print(f"Hyperparameter optimization time: {hyper_timeout/60} minutes")
print(f"Time limit: {time_limit/60} minutes")

# Custom hyperparameter tuning configuration
hyperparameter_tune_kwargs = {
    'num_trials': 20,  # Number of trials to run
    'scheduler': 'local',  # Scheduler to use for parallel training
    'searcher': 'bayes',  # Searcher to use for hyperparameter optimization
    'time_out': hyper_timeout,  # Time limit in seconds for each call to the ML model
}

predictor_new_hpo = TabularPredictor(
    label='count',
    path='autogluon-new-hpo',
    eval_metric='root_mean_squared_error'    
)

predictor_new_hpo.fit(
    train_val_df,
    time_limit=time_limit,
    presets='best_quality',
    hyperparameters=hyperparameters,
    hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
    num_cpus=6,
    num_gpus=1,
    num_stack_levels=3,
    verbosity=1
)

/Users/samuel/anaconda3/envs/bikeshare/lib/python3.11/site-packages/autogluon/tabular/predictor/predictor.py:1242: UserWarning: Failed to use ray for memory safe fits. Falling back to normal fit. Error: ValueError('ray==2.31.0 detected. 2.10.0 <= ray < 2.11.0 is required. You can use pip to install certain version of ray `pip install ray==2.10.0` ')
  stacked_overfitting = self._sub_fit_memory_save_wrapper(
Specified total num_gpus: 1, but only 0 are available. Will use 0 instead


Hyperparameter optimization time: 1.0 minutes
Time limit: 3.0 minutes


  5%|▌         | 1/20 [00:53<17:02, 53.81s/it]
Specified total num_gpus: 1, but only 0 are available. Will use 0 instead
 10%|█         | 2/20 [00:46<07:01, 23.41s/it]
Specified total num_gpus: 1, but only 0 are available. Will use 0 instead
  5%|▌         | 1/20 [00:48<15:12, 48.00s/it]
Specified total num_gpus: 1, but only 0 are available. Will use 0 instead
  5%|▌         | 1/20 [00:53<17:04, 53.90s/it]
Specified total num_gpus: 1, but only 0 are available. Will use 0 instead
 10%|█         | 2/20 [00:45<06:53, 22.99s/it]
Specified total num_gpus: 1, but only 0 are available. Will use 0 instead
  5%|▌         | 1/20 [00:50<16:07, 50.90s/it]


In [ ]:
predictor_new_hpo.leaderboard(silent=True)

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-34.082446,root_mean_squared_error,0.436478,86.309923,0.000210,0.014114,2,True,8
1,WeightedEnsemble_L5,-34.082446,root_mean_squared_error,0.436618,86.303996,0.000349,0.008187,5,True,9
2,LightGBM_BAG_L1/T2,-34.504071,root_mean_squared_error,0.078709,30.725503,0.078709,30.725503,1,True,2
3,XGBoost_BAG_L1/T3,-35.842340,root_mean_squared_error,0.136838,19.883079,0.136838,19.883079,1,True,5
4,XGBoost_BAG_L1/T2,-36.045101,root_mean_squared_error,0.096258,12.522853,0.096258,12.522853,1,True,4
5,LightGBM_BAG_L1/T1,-37.761090,root_mean_squared_error,0.124463,23.164374,0.124463,23.164374,1,True,1
6,XGBoost_BAG_L1/T1,-39.337629,root_mean_squared_error,0.100144,13.572753,0.100144,13.572753,1,True,3
7,NeuralNetTorch_BAG_L1/T2,-77.096832,root_mean_squared_error,0.078298,35.064095,0.078298,35.064095,1,True,7
8,NeuralNetTorch_BAG_L1/T1,-113.980426,root_mean_squared_error,0.081503,15.827772,0.081503,15.827772,1,True,6


In [ ]:
performance = predictor_new_hpo.evaluate(val_df)
performance

{'root_mean_squared_error': -32.49807779184623,
 'mean_squared_error': -1056.1250601648892,
 'mean_absolute_error': -19.817492352940548,
 'r2': 0.9680029353683928,
 'pearsonr': 0.9838875781242997,
 'median_absolute_error': -11.453035354614258}

In [ ]:
predictions_new_hpo = predictor_new_hpo.predict(val_df)

In [ ]:
# Replace negative predictions with zero
predictions_new_hpo[predictions_new_hpo < 0] = 0

In [ ]:
# Calculating scores of predictions
from sklearn.metrics import mean_squared_log_error

mean_squared_log_error(val_df["count"], predictions_new_hpo)

0.1248213271145344

## - Loading the Best Model and Predicting on Test Data

In [ ]:
best_model = predictor_new_hpo.model_best
print(f"The best model is {best_model}")

The best model is WeightedEnsemble_L2


In [ ]:
saved_predictor = TabularPredictor.load('autogluon-new-hpo')
saved_predictor.leaderboard(silent=True)

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-34.082446,root_mean_squared_error,0.436478,86.309923,0.000210,0.014114,2,True,8
1,WeightedEnsemble_L5,-34.082446,root_mean_squared_error,0.436618,86.303996,0.000349,0.008187,5,True,9
2,LightGBM_BAG_L1/T2,-34.504071,root_mean_squared_error,0.078709,30.725503,0.078709,30.725503,1,True,2
3,XGBoost_BAG_L1/T3,-35.842340,root_mean_squared_error,0.136838,19.883079,0.136838,19.883079,1,True,5
4,XGBoost_BAG_L1/T2,-36.045101,root_mean_squared_error,0.096258,12.522853,0.096258,12.522853,1,True,4
5,LightGBM_BAG_L1/T1,-37.761090,root_mean_squared_error,0.124463,23.164374,0.124463,23.164374,1,True,1
6,XGBoost_BAG_L1/T1,-39.337629,root_mean_squared_error,0.100144,13.572753,0.100144,13.572753,1,True,3
7,NeuralNetTorch_BAG_L1/T2,-77.096832,root_mean_squared_error,0.078298,35.064095,0.078298,35.064095,1,True,7
8,NeuralNetTorch_BAG_L1/T1,-113.980426,root_mean_squared_error,0.081503,15.827772,0.081503,15.827772,1,True,6


In [ ]:
hyper_tunning_prediction_df = saved_predictor.predict(test_df)

# Replace negative predictions with zero
hyper_tunning_prediction_df[hyper_tunning_prediction_df < 0] = 0

## - Submitting the predictions

In [ ]:
# submission
submission_hyper_tunning_df = pd.read_csv('../data/sampleSubmission.csv')
submission_hyper_tunning_df['count'] = hyper_tunning_prediction_df
submission_hyper_tunning_df.to_csv('submission-hyper-tunning.csv', index=False)

!kaggle competitions submit -c bike-sharing-demand -f submission-hyper-tunning.csv -m "hyperparameter tunning with {best_model}"

100%|█████████████████████████████████████████| 188k/188k [00:00<00:00, 381kB/s]
Successfully submitted to Bike Sharing Demand

# Including Custom Models on Hyperparameter Tuning

In [1]:
import sys
sys.path.append('..')

from src import DataLoader

## - Loading the data

In [2]:
loader = DataLoader()


        Data Loader initialized with data path: ../data/
            - Use load_raw() to load raw data
            - Use get_train_test_data() to get raw train and test data  
            - Use save_feature_engineered_data() to save feature engineered data
            - Use load_feature_engineered() to load feature engineered data
              
            - Use set_as_category() to set columns as category

        Checkpoints available: ['hyperparameter_tuning']
        


In [3]:
loader.load_feature_engineered(checkpoint_name='hyperparameter_tuning')
loader.set_as_category(columns=["season", "weather"])

train_df, test_df = loader.get_train_test_data()

Columns ['season', 'weather'] set as category successfully!


## - Creating Validation Set for Local Scoring

In [4]:
from sklearn.model_selection import train_test_split

train_val_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

print(f""" 
Train shape: {train_val_df.shape}
Validation shape: {val_df.shape}  
Test shape: {test_df.shape}  
""")

train_val_df.head()

 
Train shape: (8708, 12)
Validation shape: (2178, 12)  
Test shape: (6493, 11)  



,season,holiday,workingday,weather,temp,humidity,windspeed,count,day_of_week,hour,minute,date
2815,3,0,1,1,27.88,83,6.0032,35,2,5,0,2011-07-06
8695,3,0,0,1,36.90,39,19.9995,450,5,16,0,2012-08-04
8406,3,0,1,1,32.80,41,16.9979,276,2,15,0,2012-07-11
1543,2,0,0,2,14.76,93,7.0015,5,6,4,0,2011-04-10
4952,4,0,0,1,13.12,45,16.9979,200,5,10,0,2011-11-19


## Applying Random Forest Regressor

### - Grid Search

In [5]:
# Import random forest regressor, grid search and random search
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

model = RandomForestRegressor(random_state=42)

param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(
    model,
    param_grid,
    cv=5,
    n_jobs=-1,
    verbose=2
)

grid_search.fit(train_val_df.drop(columns=['count', 'date']), train_val_df['count'])

print(f"Best parameters: {grid_search.best_params_}")

Fitting 5 folds for each of 225 candidates, totalling 1125 fits
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.2s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.2s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.2s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.3s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.2s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   2.4s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   2.4s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   2.5s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   2.4s
[CV] END max_depth=10, min_

/Users/samuel/anaconda3/envs/bikeshare/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   9.0s
[CV] END max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   9.0s
[CV] END max_depth=30, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   3.0s
[CV] END max_depth=30, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   3.0s
[CV] END max_depth=30, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   3.0s
[CV] END max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   8.9s
[CV] END max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   9.0s
[CV] END max_depth=30, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   3.0s
[CV] END max_depth=30, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   3.0s
[CV] END max_depth=30, min_samples_leaf=1, min_samples_split=5, n_estimators=300; total tim

In [6]:
# Predict on the validation set
predictions = grid_search.predict(val_df.drop(columns=['count', 'date']))

# Replace negative predictions with zero
predictions[predictions < 0] = 0

# Calculating scores of predictions
mean_squared_log_error(val_df["count"], predictions), r2_score(val_df["count"], predictions)

NameError: name 'mean_squared_log_error' is not defined

### - Random Search

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

model = RandomForestRegressor(random_state=42)

param_distributions = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

random_search = RandomizedSearchCV(
    model,
    param_distributions,
    n_iter=20,
    cv=5,
    n_jobs=-1,
    verbose=2
)

random_search.fit(train_val_df.drop(columns=['count', 'date']), train_val_df['count'])

print(f"Best parameters: {random_search.best_params_}")

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END max_depth=30, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   2.7s
[CV] END max_depth=30, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   2.7s
[CV] END max_depth=30, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   2.7s
[CV] END max_depth=30, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   2.1s
[CV] END max_depth=30, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   2.1s
[CV] END max_depth=20, min_samples_leaf=2, min_samples_split=2, n_estimators=400; total time=   5.6s
[CV] END max_depth=20, min_samples_leaf=2, min_samples_split=2, n_estimators=400; total time=   5.7s
[CV] END max_depth=20, min_samples_leaf=2, min_samples_split=2, n_estimators=400; total time=   5.7s
[CV] END max_depth=20, min_samples_leaf=2, min_samples_split=2, n_estimators=400; total time=   5.8s
[CV] END max_depth=20, min_sa

In [ ]:
# Predict on the validation set
predictions = random_search.predict(val_df.drop(columns=['count', 'date']))

# Replace negative predictions with zero
predictions[predictions < 0] = 0

# Calculating scores of predictions
mean_squared_log_error(val_df["count"], predictions), r2_score(val_df["count"], predictions)

(0.17183809433125152, 0.8607013620248639)

# Applying Personal Cross Validated SARIMAX Model Wrapper
> https://github.com/samlexrod/cvsarimax.git

# END OF NOTEBOOK